# 1. define eager graph

In [1]:
import eagerx
from typing import Dict
import numpy as np
from huggingface_sb3 import load_from_hub
import stable_baselines3 as sb3

from double_pendulum.objects import Double_Pendulum

In [2]:
# noinspection JupyterPackage
rate = 50.0
graph = eagerx.Graph.create()
sensors = ["theta", "theta_dot", "image"]
actuators = ["u"]
states = ["model_state"]
pendulum = Double_Pendulum.make("double_pendulum", rate=rate, actuators=actuators, sensors=sensors, states=states, render_fn="double_pendulum_render_fn")

from double_pendulum.processor import DecomposedAngle,DecomposedAngle_vel
pendulum.sensors.theta.processor = DecomposedAngle.make()
pendulum.sensors.theta.space.low = -1
pendulum.sensors.theta.space.high = 1
pendulum.sensors.theta.space.shape = [4]
pendulum.sensors.theta_dot.processor = DecomposedAngle_vel.make()
pendulum.sensors.theta_dot.space.low = -999
pendulum.sensors.theta_dot.space.high = 999
pendulum.sensors.theta_dot.space.shape = [2]


graph.add(pendulum)

# Connect the pendulum to an action and observations
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle", window=60)
graph.connect(source=pendulum.sensors.theta_dot, observation="angular_velocity")

# Render image
graph.render(source=pendulum.sensors.image, rate=rate)
Double_Pendulum.info()

   entity_type: `Double_Pendulum`
   module: `double_pendulum.objects`
   file: `/home/marunyu/study/eagerx_sideproject/double_pendulum/objects.py`

Supported engines:
 - eagerx_ode.engine/OdeEngine

Make this spec with:
   spec = Double_Pendulum.make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 50.0, render_shape: List[int] = None, render_fn: str = None)

class Double_Pendulum:
   make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 50.0, render_shape: List[int] = None, render_fn: str = None):
      sensors:
       - theta: Space(-999.0, 999.0, (), float32)
       - theta_dot: Space(-999.0, 999.0, (), float32)
       - image: Space(uint8)
       - u_applied: Space([-4.], [4.], (1,), float32)
      actuators:
       - u: Space([-4.], [4.], (1,), float32)
      engine_states:
       - model_state: Space([-3.14 -3.14 -9.   -9.  ], [3.14 3.14 9.   9.  ], (4,), float32)


engine and train environment

In [3]:
from eagerx_ode.engine import OdeEngine
from double_pendulum.double_pendulum_env import Double_PendulumEnv
from gym.wrappers.rescale_action import RescaleAction
ode_engine = OdeEngine.make(rate=rate)
train_env = Double_PendulumEnv(name="train", rate=rate, graph=graph, engine=ode_engine, eval=False)
test_env = Double_PendulumEnv(name="test", rate=rate, graph=graph, engine=ode_engine, eval=True)
print("action_space: ", train_env.action_space)
print("observation_space: ", train_env.observation_space)
# ode_render = pendulum.gui(OdeEngine)
from eagerx.wrappers import Flatten
from stable_baselines3.common.env_checker import check_env
train_env = Flatten(train_env)
test_env = Flatten(test_env)

[WARN]: Backend 'SINGLE_PROCESS' does not support multiprocessing, so all nodes are launched in the ENVIRONMENT process.
action_space:  Dict(voltage:Space([-4.], [4.], (1,), float32))
observation_space:  Dict(angle:Box([[-1. -1. -1. -1.]], [[1. 1. 1. 1.]], (1, 4), float32), angular_velocity:Box([[-999. -999.]], [[999. 999.]], (1, 2), float32))


# 2 train

# 2.1 SAC

In [ ]:
sac_model = sb3.SAC("MlpPolicy", train_env, verbose=1, learning_rate=7e-4, tensorboard_log="./tensorboard/sac_doupen_tensorboard/")
train_env.render("human")
sac_model.learn(total_timesteps=int(10000))
train_env.close()
sac_model.save("./model/double_pendulum_sac")

In [4]:
from stable_baselines3.common.evaluation import evaluate_policy
import helper
# sac_model = sb3.SAC.load("./model/double_pendulum_sac.zip")
sac_model = sb3.SAC.load("./double_pendulum_sac.zip")
# mean_reward, std_reward = evaluate_policy(sac_model, test_env, n_eval_episodes=10, render=True)
# print("mean_reward:",mean_reward,"std_reward:",std_reward)
helper.evaluate(sac_model,test_env,n_eval_episodes=5,episode_length=270, video_rate=rate,video_prefix="trained_disc")

Start evaluation episode 0 of 5


  1%|          | 3/270 [00:00<00:47,  5.60it/s]QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the object's thread (0x7f950c1d25f0).
Cannot move to target thread (0x7f950c001c20)

QObject::moveToThread: Current thread (0x7f950c001c20) is not the objec

vel_cost:   33.25617057340145 [ 2.5781271  -0.03352065]
vel_reward:   20635.274776156984 [ 2.5781271  -0.03352065]
vel_cost:   25.334301771390248 [ 2.2497315  -0.03731069]
vel_reward:   19128.917354991867 [ 2.2497315  -0.03731069]
vel_cost:   19.14327517836619 [ 1.9558152  -0.02933409]
vel_reward:   22437.4888319981 [ 1.9558152  -0.02933409]
vel_cost:   14.431316132833206 [ 1.6983376  -0.02186765]
vel_reward:   26051.15100469698 [ 1.6983376  -0.02186765]
vel_cost:   10.822271802698564 [ 1.471073   -0.00998119]
vel_reward:   33042.38306327408 [ 1.471073   -0.00998119]
vel_cost:   8.115788770387878 [ 1.2740309e+00 -8.6011167e-04]
vel_reward:   39654.82670964153 [ 1.2740309e+00 -8.6011167e-04]
vel_cost:   6.182516924765747 [ 1.1117887 -0.0103581]
vel_reward:   32794.24005421202 [ 1.1117887 -0.0103581]
vel_cost:   4.744603841474526 [ 0.9731455  -0.02184395]
vel_reward:   26063.498448259685 [ 0.9731455  -0.02184395]
vel_cost:   3.664664373379232 [ 0.8536986  -0.03213859]
vel_reward:   21213

 78%|███████▊  | 210/270 [00:01<00:00, 160.04it/s]

vel_cost:   4.191140636832569 [-0.9028089  -0.07609896]
vel_reward:   8806.020494287723 [-0.9028089  -0.07609896]
vel_cost:   5.645092722828995 [-1.0361751  -0.11764315]
vel_reward:   3836.4672373148273 [-1.0361751  -0.11764315]
vel_cost:   7.633161399409301 [-1.2065902  -0.13301544]
vel_reward:   2821.0414596956834 [-1.2065902  -0.13301544]
vel_cost:   14.195108558390785 [-1.605615    0.25545162]
vel_reward:   243.74870413296867 [-1.605615    0.25545162]
vel_cost:   28.93611514261579 [-2.0277615  0.6471875]
vel_cost:   18.036819264987347 [-1.8246824  -0.26358023]
vel_reward:   207.17534848518147 [-1.8246824  -0.26358023]
vel_cost:   26.13083596086952 [-1.961311  -0.5872449]
vel_cost:   54.40867679001208 [-1.9380966 -1.3346832]
vel_cost:   73.07089167084087 [-2.2514925 -1.5447459]


...

vel_cost:   34.53665611131334 [ 2.6279325  -0.01803984]
vel_reward:   28123.856296011298 [ 2.6279325  -0.01803984]
vel_cost:   26.449646916429298 [ 2.2994025  -0.02587296]
vel_reward:   24045.68993424836 [ 2.2994025  -0.02587296]
vel_cost:   20.031620681092743 [ 2.0014377  -0.01195213]
vel_reward:   31765.233249353638 [ 2.0014377  -0.01195213]
vel_cost:   15.266931875556617 [ 1.7472526  -0.01112188]
vel_reward:   32297.097846876455 [ 1.7472526  -0.01112188]
vel_cost:   11.555458868891893 [1.5202171  0.00282228]
vel_reward:   38128.776120534196 [1.5202171  0.00282228]
vel_cost:   8.81893805041441 [1.3279966  0.00729057]
vel_reward:   34869.185296292344 [1.3279966  0.00729057]
vel_cost:   6.87233808802868 [ 1.1722814  -0.00748287]
vel_reward:   34735.33794201226 [ 1.1722814  -0.00748287]
vel_cost:   5.393408925551448 [ 1.0380374  -0.01703013]
vel_reward:   28697.56634482894 [ 1.0380374  -0.01703013]
vel_cost:   4.312152704255525 [ 0.9267454  -0.02988928]
vel_reward:   22189.722538549468 

100%|██████████| 270/270 [00:01<00:00, 158.02it/s]

vel_cost:   32.620082324815144 [ 2.5539224  -0.01934402]
vel_reward:   27399.767598762508 [ 2.5539224  -0.01934402]
vel_cost:   24.323348310677268 [ 2.2055705  -0.00567034]
vel_reward:   36017.61619661313 [ 2.2055705  -0.00567034]
vel_cost:   18.26759258161178 [ 1.9109695  -0.02070086]
vel_reward:   26666.222553806234 [ 1.9109695  -0.02070086]
vel_cost:   13.41686294938687 [ 1.6380115  -0.00853026]
vel_reward:   34015.274954843844 [ 1.6380115  -0.00853026]
vel_cost:   9.870134394767513 [ 1.4047476  -0.01333271]
vel_reward:   30900.14169475529 [ 1.4047476  -0.01333271]
vel_cost:   7.337193956590903 [ 1.2089657  -0.03821265]
vel_reward:   18786.93941646684 [ 1.2089657  -0.03821265]
vel_cost:   5.4190073818814 [ 1.0344725  -0.05845553]
vel_reward:   12532.238076934957 [ 1.0344725  -0.05845553]
vel_cost:   3.8829154691175205 [ 0.8721359  -0.06317061]
vel_reward:   11404.43904863166 [ 0.8721359  -0.06317061]
vel_cost:   2.7173263003915347 [ 0.7253759  -0.06575537]
vel_reward:   10829.863016

Showing episode 4 with episodic reward: 997514.6766766048


Finished evaluation with mean episodic reward: 1250693.11945621


# 2.2 DDPG

In [ ]:
ddpg_model = sb3.DDPG("MlpPolicy", train_env, verbose=1, learning_rate=5e-4, tensorboard_log="./tensorboard/ddpg_doupen_tensorboard/")
train_env.render("human")
ddpg_model.learn(total_timesteps=int(10000))
train_env.close()
ddpg_model.save("./model/double_pendulum_ddpg")

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
ddpg_model = sb3.DDPG.load("./model/double_pendulum_ddpg.zip")
mean_reward, std_reward = evaluate_policy(ddpg_model, test_env, n_eval_episodes=10, render=True)
print("mean_reward:",mean_reward,"std_reward:",std_reward)

# 2.3 PPO

In [ ]:
ppo_model = sb3.PPO("MlpPolicy", train_env, verbose=1, learning_rate=5e-4, tensorboard_log="./tensorboard/ppo_doupen_tensorboard/")
train_env.render("human")
ppo_model.learn(total_timesteps=int(10000))
train_env.close()
ddpg_model.save("./model/double_pendulum_ppo")

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
ppo_model = sb3.DDPG.load("./model/double_pendulum_ppo.zip")
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, n_eval_episodes=10, render=True)
print("mean_reward:",mean_reward,"std_reward:",std_reward)